In [4]:
import os
import base64
import matplotlib.pyplot as plt
from mistralai import Mistral
import json
import pandas as pd
from typing import Union
import numpy as np
import secrets
import time

pd.set_option('display.max_colwidth', None)
# Load Mistral API key from environment variables
api_key = "ZL4LdkeCwsoIYcNx2aFDujTYgpwj2kij"
# Initialize the Mistral client
client = Mistral(api_key=api_key)

In [ ]:
def switch_cle():
    key = np.random.choice(["G8in7nx1BvV5jwFZeb7oYUOi9OQaKsvD","qAL87sEMQCSGKcO58179DXyH3jedFPX2","9tlWzQ18hIo6Zl9x68j99PM3J4hDgr9L"])
    client = Mistral(api_key=key)
    return client

In [33]:
def img_to_base64(image_path):
    """Input : image_path (str) : path to the image file
    Returns : image_base64 (str) : base64 encoded image"""
    
    with open(image_path, 'rb') as image_file:
        image_bytes = image_file.read()

    image_base64 = base64.b64encode(image_bytes).decode('utf-8')
    
    return image_base64

def json_to_dataframe(json_data: Union[str, dict], key: str = None) -> pd.DataFrame:
    # If json_data is a string, parse it into a dictionary
    if isinstance(json_data, str):
        json_data = json.loads(json_data)
    
    # If a key is provided, extract the list of records from the JSON object
    if key is not None:
        data = json_data[key]
    else:
        data = json_data
    
    # Convert the list of records to a pandas DataFrame
    df = pd.DataFrame(data)
    
    return df

def list_clothes(args): 
    client=switch_cle()
    image_base64, guide = args
    time.sleep(1)
    # Define the messages for the chat API
    messages = [
        {
            "role": "system",
            "content": "Return the answer in a JSON object with the next structure: "
                    "{\"elements\": [{\"element\": \"some name for element1\", "
                    "\"color\": \"the color of element1\", "
                    "\"fit\": \"the fit, shape of element1. Be concise.\", "
                    "\"price\": \"some number, estimated price of element1\", "
                    "\"context\": \"a word describing the occasion, mood or functionality of the piece\", "
                    "\"description\": \"a description of element1, emphasizing on the vibe of the piece and that encapsulates the precedent variables\"}, "
                    "{\"element\": \"some name for element2\", ...}]}"
        },
        {
            "role": "system",
            "content": f"You are a fashion critique, neutral and objective.\n\n {guide} \n\n You are presented with an image of an outfit, describe each of the elements thanks to your expertise "
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": f"data:image/jpeg;base64,{image_base64}"
                }
            ]
        }
    ]

    # Call the Mistral API to complete the chat
    chat_response = client.chat.complete(
        model="pixtral-12b-2409",
        messages=messages,
        temperature=0.4,
        response_format={
            "type": "json_object",
        }
    )

    # Get the content of the response
    content = chat_response.choices[0].message.content
    
    return content

def reformulate_list_clothes(args): 
    client=switch_cle()
    clothes = args
    time.sleep(1)
    # Define the messages for the chat API
    messages = [
        {
            "role": "system",
            "content": "Return the answer in a JSON object with the next structure: "
                    "{\"elements\": [{\"element\": \"some name for element1\", "
                    "\"color\": \"the color of element1\", "
                    "\"fit\": \"the fit, shape of element1\", "
                    "\"price\": \"some number, estimated price of element1\", "
                    "\"context\": \"a word describing the occasion, mood or functionality of the piece\", "
                    "\"description\": \"a description of element1, emphasizing on the vibe of the piece and that encapsulates the precedent variables\"}, "
                    "{\"element\": \"some name for element2\", ...}]}"
        },
        {
            "role": "system",
            "content": f"Reformulate the list of clothes, keeping the same meaning an exact same structure. Just change the wording."
        },
        {
            "role": "user",
            "content": clothes
        }
    ]

    # Call the Mistral API to complete the chat
    chat_response = client.chat.complete(
        model="pixtral-12b-2409",
        messages=messages,
        temperature=0.4,
        response_format={
            "type": "json_object",
        }
    )

    # Get the content of the response
    content = chat_response.choices[0].message.content
    
    return content

def process_critique(args):
    client=switch_cle()
    time.sleep(1)
    outfit, guide = args
    
    chat_response = client.chat.complete(
        model="mistral-large-latest",
        messages=[
            {"role": "system", "content": "As a 'Fashion Critique', your mission is to help relook people. \n\n {guide} \n\n You are given a description of items in an outfit. Give a critique of the outfit, outlining the general vibe, how the pieces work together and what could be improved."},
            {"role": "user", "content": outfit},
        ],
        temperature=0.2,
        max_tokens=2048
    )
    result = chat_response.choices[0].message.content

    return result

def recommend_item(args):
    client=switch_cle()
    time.sleep(1) 
    critique, guide, color_rule, piece_rule = args  
    # Define the messages for the chat API
    messages = [
        {
            "role": "system",
            "content": "Return the answer in a JSON object with the next structure: "
                    "{\"elements\": [{\"element\": \"some short name for element1\", "
                    "\"color\": \"the color of element1\", "
                    "\"fit\": \"the fit, shape of element1\", "
                    "\"price\": \"some number, estimated price of element1\", "
                    "\"context\": \"a word describing the occasion, mood or functionality of the piece\", "
                    "\"description\": \"a description of element1, emphasizing on the vibe of the piece and that encapsulates the precedent variables\"}]}"
        },
        {
            "role": "system",
            "content": f"You are a fashion critique, neutral and objective.\n\n {guide} \n\n You are presented with a critique of an outfit, describe a single element that would improve the outfit."
        },
        {
            "role": "system",
            "content": "Follow this rule when suggesting a piece of clothing: " + piece_rule
        },
        {
            "role": "system",
            "content": "Follow this rule when choosing a color: " + color_rule
        },
        {
            "role": "user",
            "content": critique
        }
    ]

    # Call the Mistral API to complete the chat
    chat_response = client.chat.complete(
        model="mistral-large-latest",
        messages=messages,
        temperature=0.4,
        response_format={
            "type": "json_object",
        }
    )

    # Get the content of the response
    content = chat_response.choices[0].message.content
    
    return content

In [7]:
desc_guide_path = "./desc_guide.txt"
critique_guide_path = "./critique_guide.txt"

with open(critique_guide_path, "rb") as f:
    critique_guide = f.read()

with open(desc_guide_path, "r") as f:
    desc_guide = f.read()

In [34]:
n_images=500
all_images = os.listdir("C:/Users/bapti/Documents/DATA/Cours/3A/Hackathon/fashion_images_dataset/")
n_images = np.random.choice(all_images, n_images, replace=False)
print(n_images)

['1217.jpg' '0919.jpg' '1095.jpg' '1408.jpg' '0736.jpg' '1078.jpg'
 '1549.jpg' '1371.jpg' '1687.jpg' '0789.jpg' '1995.jpg' '1194.jpg'
 '0999.jpg' '0675.jpg' '1442.jpg' '0454.jpg' '1854.jpg' '0477.jpg'
 '0990.jpg' '1609.jpg' '0352.jpg' '1647.jpg' '0262.jpg' '0710.jpg'
 '0804.jpg' '0167.jpg' '1076.jpg' '1940.jpg' '0197.jpg' '1605.jpg'
 '0903.jpg' '0367.jpg' '1186.jpg' '0272.jpg' '1503.jpg' '0369.jpg'
 '1873.jpg' '0447.jpg' '0296.jpg' '0759.jpg' '1788.jpg' '2081.jpg'
 '1744.jpg' '1238.jpg' '1354.jpg' '0925.jpg' '1407.jpg' '0661.jpg'
 '1017.jpg' '0790.jpg' '0140.jpg' '1025.jpg' '2043.jpg' '1099.jpg'
 '1037.jpg' '1064.jpg' '0221.jpg' '1373.jpg' '1997.jpg' '0875.jpg'
 '1749.jpg' '0997.jpg' '0409.jpg' '0650.jpg' '0870.jpg' '1482.jpg'
 '0199.jpg' '0149.jpg' '1785.jpg' '1270.jpg' '0145.jpg' '0491.jpg'
 '1750.jpg' '0042.jpg' '0720.jpg' '0485.jpg' '2077.jpg' '0678.jpg'
 '1204.jpg' '1126.jpg' '1760.jpg' '1403.jpg' '0277.jpg' '1276.jpg'
 '1502.jpg' '1738.jpg' '0102.jpg' '1684.jpg' '1310.jpg' '1671.

In [35]:
from concurrent.futures import ThreadPoolExecutor
    
args_clothes = [(img_to_base64("C:/Users/bapti/Documents/DATA/Cours/3A/Hackathon/fashion_images_dataset/"+img), desc_guide) for img in n_images]
Reformulations = []

# Using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=8) as executor:
    # Submit tasks and collect results
    Reformulations.append(list(executor.map(list_clothes, args_clothes)))

In [ ]:
# Using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit tasks and collect results
    Reformulations.append(list(executor.map(reformulate_list_clothes, Reformulations[0])))
    
# Using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit tasks and collect results
    Reformulations.append(list(executor.map(reformulate_list_clothes, Reformulations[1])))

In [36]:
Clothes_df = [json_to_dataframe(clothes, key='elements') for clothes in Reformulations[0]]
args_critiques = [('-'+'\n-'.join(clothes['description']),critique_guide) for clothes in Clothes_df]

with ThreadPoolExecutor(max_workers=8) as executor:
    # Submit tasks and collect results
    Critiques = list(executor.map(process_critique, args_critiques))

SDKError: API error occurred: Status 502
<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>


In [ ]:
n_suggest = 1
color_rules = np.array([np.random.choice([ 
                            "complementary colors: colors that are opposite on the color wheel", 
                            "analogus colors: colors that are close on the color wheel", 
                            "accent color: one bright color that pops from the rest that are neutral", 
                            "sandwiching: layering a bright color between two neutral colors",
                            "monochromatic: using different shades of the same color to create a cohesive look",
                            "pattern mixing: combining different patterns to create a unique outfit",
                            "seasonal: using seasonal colors and pieces to create a weather-appropriate outfit",
                                        ], n_suggest, replace=False) for i in range(len(Critiques))])
        
piece_rules = np.array([np.random.choice([
                                "mixing textures: incorporating different textures to add visual interest",
                                "statement piece: building an outfit around a bold statement piece",
                                "proportion balance: suggest a piece with a fit that balances the outfit", 
                                "accessories: adding an accessory to elevate the outfit",
                                "silouhette: creating a visually interesting shape with the outfit",
                                "replacement: suggesting a piece that would replace a current piece in the outfit",
                                "layering: adding a layer to the outfit to create depth, like a coat or jacket",
                                "adding surface: suggest a pig piece with a different surface, like a shiny or matte fabric"
                                    ], n_suggest, replace=False) for i in range(len(Critiques))])
Suggestions = []

for i in range(n_suggest):
    print(i)
    args_reco = [(Critiques[j], desc_guide, color_rules[j][i], piece_rules[j][i]) for j in range(len(Critiques))]
    
    with ThreadPoolExecutor(max_workers=5) as executor:
    # Submit tasks and collect results
        Recos = list(executor.map(recommend_item, args_reco))
    
    Suggestions.append(Recos)

0


In [ ]:
Desc = []
Reco = []
for i in range(n_suggest):
    for j in range(len(Critiques)):
        Desc.append(Reformulations[i][j])
        Reco.append(Suggestions[i][j])

In [ ]:
random_hash = secrets.token_hex(4)
with open(f"./data/descriptions_{random_hash}.json", "w") as file:
    json.dump(Desc, file)
    
with open(f"./data/recommendations_{random_hash}.json", "w") as file:
    json.dump(Reco, file)

In [23]:
with open(f"./data/descriptions_{random_hash}.json", "r") as file:
    loaded_desc = json.load(file)
    
with open(f"./data/recommendations_{random_hash}.json", "r") as file:
    loaded_reco = json.load(file)

In [24]:
d_df=pd.concat([json_to_dataframe(loaded_desc[i],key='elements') for i in range(len(loaded_desc))])
r_df=pd.concat([json_to_dataframe(loaded_reco[i],key='elements') for i in range(len(loaded_reco))])

In [30]:
r_df.sample(15)

,element,color,fit,price,context,description
0,Structured Leather Tote,Black,"Structured, with a wide base and two handles",250.0,Professional,"A structured leather tote in black that complements the monochromatic palette and adds a touch of sophistication. Its wide base and two handles make it practical for carrying essentials, while its sleek design maintains the outfit's professional vibe. The tote balances the wide-leg trousers and creates a harmonious silhouette, adhering to the proportion balance rule."
0,Leather Jacket,Purple,"Fitted, with a slight crop to emphasize the waist",350.0,Edgy,"A fitted purple leather jacket with a slight crop to emphasize the waist, adding a layer of depth and edge to the outfit. This jacket complements the yellow top while maintaining the bold, urban chic vibe of the look. Its sleek texture balances the feathered coat's drama, making the outfit more cohesive and balanced. Perfect for a creative or artsy setting, this jacket is a statement piece that exudes confidence and style. Price estimated at $350."
0,Long Pendant Necklace,Gold,Adjustable length chain with a delicate pendant,45.0,Elevates,"A long pendant necklace in gold, with an adjustable length chain and a delicate pendant. This piece will add a touch of elegance and bohemian charm to the outfit, helping to tie the rugged denim shorts and edgy leather vest together. The gold color complements the existing analogous color palette of the outfit, creating a more cohesive and stylish look. The necklace will elevate the outfit, making it more put-together and balanced."
0,Belt,Navy,"Adjustable, to cinch the waist",45.0,Professional,"A slim, navy leather belt with a sleek gold buckle to cinch the waist, adding definition to the silhouette and a touch of elegance to the outfit. The belt complements the navy stripes in the heels and the blue peplum skirt, creating a cohesive monochromatic look that exudes quiet sophistication and polish. Perfect for a business casual setting, this belt is both functional and stylish, enhancing the overall professional vibe of the outfit."
0,Leather Jacket,"Black, with a white pinstripe pattern on the sleeves","Slim fit, cropped at the waist with a asymmetrical zipper",250.0,edgy,A sleek black leather jacket with a white pinstripe pattern on the sleeves for a twist on pattern mixing. The slim fit and cropped waist with an asymmetrical zipper add an edgy touch while maintaining a flattering silhouette. Perfect for layering over the burgundy dress to create depth and enhance the urban vibe of the outfit.
0,Clutch,Metallic,Structured,50.0,Evening,"A metallic clutch that introduces a pop of contrast and complements the overall elegance of the outfit, perfect for evening events."
0,red beret,red,one size,25.0,casual,"A vibrant red beret to add a playful pop of color to the outfit, emphasizing the retro and confident vibe while elevating the look with a stylish French-inspired touch. The bold accent color will complement the neutral accessories and bring out the vibrant tones in the tartan co-ord set."
0,White Leather Sneakers,White,"Lace-up, round toe with a slim sole",80.0,"Casual yet polished, perfect for urban exploration","A pair of white leather sneakers with a slim sole and lace-up detail. These sneakers offer a more elevated look compared to the current sporty sneakers, while still maintaining comfort. The white color sandwiching between the black tank top and the bold patterned shorts creates a harmonious transition, enhancing the overall color balance of the outfit. Ideal for a day of leisure or errands in an urban setting, these sneakers provide a casual yet polished touch to the ensemble."
0,Red Scarf,Red,One size fits all,25.0,Casual,"A vibrant red scarf to add a pop of complementary color to the monochromatic outfit. This scarf brings warmth and a personal touch to the urban street-style vibe, making the look more eye-catching and unique. It's perfect for cooler days and adds just the right amount of 